In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk

In [67]:
stemmer = SnowballStemmer("russian")

In [68]:
def stem_tokenize(text):
    tokens = word_tokenize(text, language="russian")
    stems = [stemmer.stem(token) for token in tokens]
    return stems

In [69]:
texts = [
    "это отличный фильм", 
    "это ужасное качество", 
    "это сегодня солнечная погода", 
    "это обычный продукт", 
    "от этого фильма не ожидал такого плохого конца это очень плохо",
    "это восхитительный фильм"
]
labels = [1, 0, 2, 2, 0, 1]

Создадим пайплаин:
1) 'vectorizer' – преобразует текст в числовые признаки (TF-IDF матрицу).
2) 'classifier' – обучает модель SVM на этих признаках.

Схема TF-IDF: 
* TF (Term Frequency) - как часто слово встречается в документе
* IDF (Inverse Document Frequency) - насколько слово уникально для всего набора документов



In [70]:
model = Pipeline([
    ('vectorizer', TfidfVectorizer(
        tokenizer=stem_tokenize,
        token_pattern=None 
    )),
    ('classifier', SVC(kernel='linear')) 
])

In [83]:
model.fit(texts, labels)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function stem_tokenize at 0x7fda424bfb00>)),
                ('classifier', SVC(kernel='linear'))])

Немного подробнее про TF-IDF:
Расчёт Term Frequency (TF)
```
TF(word, doc) = (количество вхождений word в doc) / (общее количество слов в doc)
```
 Расчёт Document Frequency (DF)
```
DF(word) = количество документов, содержащих word
```
 Расчёт Inverse Document Frequency (IDF)
```
IDF(word) = log(общее количество документов / DF(word)) + 1
```
Расчёт TF-IDF
```
TF-IDF(word, doc) = TF(word, doc) * IDF(word)
```




In [72]:
vectorizer = model.named_steps['vectorizer']

tfidf_matrix = vectorizer.transform(texts)

import pandas as pd

feature_names = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(), 
    columns=feature_names,
    index=[f"Док {i+1}" for i in range(len(texts))]
)
print(df_tfidf)

       восхитительн   качеств      конц        не     обычн     ожида  \
Док 1      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Док 2      0.000000  0.674653  0.000000  0.000000  0.000000  0.000000   
Док 3      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Док 4      0.000000  0.000000  0.000000  0.000000  0.674653  0.000000   
Док 5      0.000000  0.000000  0.297911  0.297911  0.000000  0.297911   
Док 6      0.772358  0.000000  0.000000  0.000000  0.000000  0.000000   

             от    отличн      очен      плох     погод   продукт    сегодн  \
Док 1  0.000000  0.772358  0.000000  0.000000  0.000000  0.000000  0.000000   
Док 2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Док 3  0.000000  0.000000  0.000000  0.000000  0.559275  0.000000  0.559275   
Док 4  0.000000  0.000000  0.000000  0.000000  0.000000  0.674653  0.000000   
Док 5  0.297911  0.000000  0.297911  0.595822  0.000000  0.000000  0.000000   
Док 6  0.00000

In [81]:
test_phrases = [
    "это очень плохой фильм",  
    "нормальный день",    
    "это восхитительно"       
]

In [82]:
predictions = model.predict(test_phrases)

for phrase, pred in zip(test_phrases, predictions):
    print(f"'{phrase}' → {pred}")

'это очень плохой фильм' → 0
'нормальный день' → 2
'это восхитительно' → 1


Простая реализация построение TF-IDF матрицы  

In [75]:
import math
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer

tokenized_docs = [stem_tokenize(doc) for doc in texts]
print("Токенизированные документы:")
for i, doc in enumerate(tokenized_docs):
    print(f"Док {i+1}: {doc}")

vocabulary = sorted(set(word for doc in tokenized_docs for word in doc))
print("\nСловарь:", vocabulary)

def compute_tf(docs):
    tf = []
    for doc in docs:
        doc_tf = {}
        doc_length = len(doc)
        for word in vocabulary:
            doc_tf[word] = doc.count(word) / doc_length if doc_length > 0 else 0
        tf.append(doc_tf)
    return tf

tf_scores = compute_tf(tokenized_docs)
print("\nTF (Term Frequency):")
for i, doc_tf in enumerate(tf_scores):
    print(f"Док {i+1}:")
    for word in vocabulary:
        if doc_tf[word] > 0:
            print(f"  {word}: {doc_tf[word]:.3f}")

df = defaultdict(int)
for word in vocabulary:
    for doc in tokenized_docs:
        if word in doc:
            df[word] += 1

print("\nDF (Document Frequency):")
for word in vocabulary:
    print(f"  {word}: {df[word]}")

total_docs = len(texts)
idf = {}
for word in vocabulary:
    idf[word] = math.log(total_docs / df[word]) + 1  # +1 чтобы избежать нулей

print("\nIDF (Inverse Document Frequency):")
for word in vocabulary:
    print(f"  {word}: {idf[word]:.3f}")

tfidf = []
for doc_tf in tf_scores:
    doc_tfidf = {}
    for word in vocabulary:
        doc_tfidf[word] = doc_tf[word] * idf[word]
    tfidf.append(doc_tfidf)

print("\nTF-IDF:")
for i, doc_tfidf in enumerate(tfidf):
    print(f"Док {i+1}:")
    for word in vocabulary:
        if doc_tfidf[word] > 0:
            print(f"  {word}: {doc_tfidf[word]:.3f}")

print("\nМатрица TF-IDF:")
header = "Документ\t" + "\t".join(vocabulary)
print(header)
for i, doc_tfidf in enumerate(tfidf):
    row = f"Док {i+1}\t\t" + "\t".join([f"{doc_tfidf[word]:.3f}" for word in vocabulary])
    print(row)

Токенизированные документы:
Док 1: ['эт', 'отличн', 'фильм']
Док 2: ['эт', 'ужасн', 'качеств']
Док 3: ['эт', 'сегодн', 'солнечн', 'погод']
Док 4: ['эт', 'обычн', 'продукт']
Док 5: ['от', 'эт', 'фильм', 'не', 'ожида', 'так', 'плох', 'конц', 'эт', 'очен', 'плох']
Док 6: ['эт', 'восхитительн', 'фильм']

Словарь: ['восхитительн', 'качеств', 'конц', 'не', 'обычн', 'ожида', 'от', 'отличн', 'очен', 'плох', 'погод', 'продукт', 'сегодн', 'солнечн', 'так', 'ужасн', 'фильм', 'эт']

TF (Term Frequency):
Док 1:
  отличн: 0.333
  фильм: 0.333
  эт: 0.333
Док 2:
  качеств: 0.333
  ужасн: 0.333
  эт: 0.333
Док 3:
  погод: 0.250
  сегодн: 0.250
  солнечн: 0.250
  эт: 0.250
Док 4:
  обычн: 0.333
  продукт: 0.333
  эт: 0.333
Док 5:
  конц: 0.091
  не: 0.091
  ожида: 0.091
  от: 0.091
  очен: 0.091
  плох: 0.182
  так: 0.091
  фильм: 0.091
  эт: 0.182
Док 6:
  восхитительн: 0.333
  фильм: 0.333
  эт: 0.333

DF (Document Frequency):
  восхитительн: 1
  качеств: 1
  конц: 1
  не: 1
  обычн: 1
  ожида: 1
  о